# Dinitz's Algorithm
made by: Kamila Mosińska 247069 MMDA

-----

Dinitz's algorithm is a method for solving the maximum flow problem (MFP) in a flow network. Named after it's creator: Yefim Dinitz, a computer scientist, it was published in the 1970s and it greatly revolutionized the approach to the Maximum Flow problem.

In general flow networks (and therefore the maximum flow problem) have their use in many different scenarios across the world:

- Transportation systems: How much material can be transported from one place to another given road/vehicle capacities.
- Electrical engineering: How much current can flow through a given circuit.
- Network routing: How much data can be sent across a network while respecting bandwidth limits.
- Project scheduling: Allocation of limited resources to tasks.



------
### Methods of the algorithm

In a directed graph where edges have capacities, the goal is to determine the maximum amount of "flow" that can pass from a source vertex (s) to a sink vertex (t) without exceeding the capacities of the edges and ensuring flow conservation at intermediate nodes.

Dinitz's algorithm enhanced the MFP efficiency by introducing a layered graph approach and processing the flow in stages, by implementing both BFS and DFS:

- Using BFS it constructs a so called 'level graph' -  where nodes are assigned levels based on the shortest distance from the source node (s).
- Using DFS it augments flow in the level graph. The algorithm uses the concept of a 'blocking flow' -  the blocking flow is a flow where all paths are saturated or all augmenting paths have been exhausted in the level graph.

The algorithm finds valid paths from source to sink. Then the algorithm iterates through multiple DFS until a blocking flow is reached. It repeats the process of finding valid paths and augumenting flow until there is no valid path from source to sink. The sum of all the flows sent through the DFS's is the desired maximum flow of a given graph.



------
### Time complexity

The algorithm's complexity in the below form is: $O(V^2E)$ because:
- each BFS level graph can be constructed in $O(E)$ time
- a blocking flow can be found by multiple DFS's in at most $O(VE)$ time

So the flow of one layer can be computed in $O(E)+O(VE) = O(VE)$ time, and since the maximum number of needed BFS level graphs cannot exceed V, it brings the algorithm's total complexity to $O(V^2E)$.

-----
### The implementation:

In [35]:
class Edge:
    # initializing edges with vertex - what vertex the edge points to, flow i.e. current flow of the edge
    # capacity of the edge, and the index of reverse edge in the adj list
    def __init__(self, vertex, flow, capacity, reverse):
        self.vertex = vertex
        self.flow = flow
        self.capacity = capacity
        self.reverse = reverse
 
 
class Graph:
    # defining the initial structure of the graph
    # with adjacency list of outgoing edges, int number of vertices and a level list (which will contain vertex levels assigned by BFS)
    def __init__(self, V):
        self.adj = [[] for i in range(V)] # adjacency list of outgoing edges for each vertex
        self.V = V # nr of vertices
        self.level = [0 for i in range(V)] # level list of all vertices
 
    # add edge from vertex u to vertex v of a certain capacity
    def addEdge(self, u, v, capacity):
        # adding a forward/normal edge with 0 flow and C capacity initially
        a = Edge(v, 0, capacity, len(self.adj[v]))
 
        # reverse edge with 0 flow and 0 cap initially
        b = Edge(u, 0, 0, len(self.adj[u]))
        self.adj[u].append(a)
        self.adj[v].append(b)
 
    # this BFS traverses our graph, returning True if there is a path from source to sink
    # or false, while performing 'level ordering'
    def BFS(self, s, t):
        # marking all vertices as unvisited initally by -1
        for i in range(self.V):
            self.level[i] = -1
 
        # source level is 0
        self.level[s] = 0
 
        # the bfs queue
        # initialize bfs with the sink
        queue = []
        queue.append(s)
        # while queue is not empty, do BFS
        while queue:
            u = queue.pop(0) # popping first queue element
            for i in range(len(self.adj[u])): # looping each of its adjacent edges
                e = self.adj[u][i] # e is edge from adj list of vertex u
                # if vertex of the edge has not been visited and its flow is smaller than capacity
                if self.level[e.vertex] < 0 and e.flow < e.capacity:
 
                    # level of current vertex is parent + 1
                    # and gets added to queue to do bfs on later
                    self.level[e.vertex] = self.level[u]+1
                    queue.append(e.vertex)
                    
        # return False if sink is not reachable i.e. has still lvl -1
        return False if self.level[t] < 0 else True
 

    # function sending available flow from sink to source
    # u - source, t - sink, flow - flow sent from s to t when function is called
    # start - list of visited edges for all vertices in the graph
    def send_flow_DFS(self, u, flow, t, start):
        # sink detected
        if u == t:
            return flow
        
        # the number of visited edges of vertex u is smaller than the total number of adj edges
        while start[u] < len(self.adj[u]):
 
            # pick the next edge from adj list of vertex u
            e = self.adj[u][start[u]]

            # if the next vertex to visit is of the 'next level' 
            # and flow of the edge connecting them is smaller than capacity
            if self.level[e.vertex] == self.level[u]+1 and e.flow < e.capacity:
 
                # set the amount of flow for this pass to be the lower of
                # either the total flow set by the parent/beginning or the flow available for the current edge e
                curr_flow = min(flow, e.capacity-e.flow)
                # sending flow to the next vertex
                temp_flow = self.send_flow_DFS(e.vertex, curr_flow, t, start)
 
                # if sent flow exists and is greater than zero
                if temp_flow and temp_flow > 0:
 
                    # add flow to current edge
                    e.flow += temp_flow
 
                    # updating the flow in the reverse edge
                    self.adj[e.vertex][e.reverse].flow -= temp_flow
                    return temp_flow
            start[u] += 1 # adding + 1 to the list of visited edges
 
    # function performing the actual algorithm
    # first looks for available paths via BFS
    # then sends flow from source to sink until there is no more valid paths from source to sink
    def Dinitz_algorithm(self, s, t):
 
        # if source is sink
        if s == t:
            return -1
 
        # initializing maximum flow
        maximum_flow = 0
 
        # while bfs is still finding a path to the sink
        while self.BFS(s, t) == True:

            # initialize start which will
            # store how many edges a given vertex has visited
            start = [0 for i in range(self.V+1)]

            #initialize sending flow
            while True:
                # initialize flow to be from s to t and inf
                flow = self.send_flow_DFS(s, float('inf'), t, start) # here send flow appears only once, but is recursively called within its own function
                if not flow: # if flow not found or zero
                    break
 
                # add the newely discovered flow to our maximum_flow
                maximum_flow += flow
        return maximum_flow


-------
Example to be shown in class:

In [36]:
class_example = Graph(11)
class_example.addEdge(0, 1, 4)
class_example.addEdge(0, 2, 12)
class_example.addEdge(0, 3, 10)
class_example.addEdge(1, 4, 8)
class_example.addEdge(2, 4, 15)
class_example.addEdge(2, 5, 20)
class_example.addEdge(3, 6, 20)
class_example.addEdge(4, 7, 10)
class_example.addEdge(4, 5, 10)
class_example.addEdge(5, 8, 22)
class_example.addEdge(6, 8, 10)
class_example.addEdge(6, 9, 10)
class_example.addEdge(7, 10, 6)
class_example.addEdge(8, 9, 15)
class_example.addEdge(8, 10, 14)
class_example.addEdge(9, 10, 18)

print("Maximum flow:", class_example.Dinitz_algorithm(0, 10))

Maximum flow: 26
